https://www.health.com/diseases-conditions

In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}


In [14]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问服务器，获取cookie
#     res = requests.get('https://www.health.com/diseases-conditions', headers=headers_pc, proxies= randomproxy,)
#     # 将cookieJar数据转化为字典，s
#     cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
#     return headers_pc, randomproxy, cookie_dict
    return headers_pc, randomproxy

# 1、先获取术语列表

In [15]:
def get_term_url(url):
    results = []
    headers_pc, randomproxy = changeparams()
    try:
        requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, timeout=10)
#             encoding = cchardet.detect(r.content)['encoding']
#             html = r.content.decode(encoding)
        r.encoding = 'utf-8'
        html = r.text
        
        with open('D:/Intern_Project/6_health.com/术语列表页.html', 'w+', encoding='utf-8') as f:
             f.write(html)

        selector = Selector(text=html)
        termUrl_list = selector.xpath(
            "//a[@class='link-list__link type-title--sm text-link']")
        #print(len(termUrl_list))
        for termUrl in termUrl_list:
            url = termUrl.xpath('./@href').extract()[0]
            term = termUrl.xpath('./text()').extract()[0].strip()
            # print(term)
            tmp = {
                'term': term,
                'url': url
            }
            results.append(tmp)
            # time.sleep()
        # pbar.update(1)
    except Exception as e:
        print(e)
    return results

In [16]:
# 获取术语链接列表
health_url = f'https://www.health.com/diseases-conditions'
health_results = get_term_url(health_url)
with open('D:/Intern_Project/6_health.com/术语链接列表.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(health_results, indent=2, ensure_ascii=False))

# 2、获取每个术语对应的文章链接列表

In [9]:
# 用来测试哪些页面能简单获取到
# headers_pc, randomproxy = changeparams()
            
# requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
# s = requests.session()
# s.keep_alive = False # 关闭多余连接

# # url = f'https://www.medicinenet.com/diseases_and_conditions/alpha_u.htm'
# url = f'http://www.everydayhealth.com/'
# # r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
# r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, timeout=10)
# #             encoding = cchardet.detect(r.content)['encoding']
# #             html = r.content.decode(encoding)
# r.encoding = 'utf-8'
# html = r.text
# with open('详情页.html', 'w+', encoding='utf-8') as f:
#     f.write(html)


# selector = Selector(text=html)
# term_url_list = selector.xpath("//div[@class='AZ_results']/ul").extract()
# print(len(term_url_list))
# for term_url in term_url_list:
#     try:
#         url = term_url.xpath('./li/a/@href').extract()[0]
#         if url is None or len(url) == 0:
#             print("失败")
#             continue
#         term = term_url.xpath('./li/a/text()').extract()[0]
#         print(url)
#         print(term)
#     except Exception as e:
#         print(e)